In [5]:
import pandas as pd
from geopy.geocoders import Nominatim
from scipy.interpolate import NearestNDInterpolator
import os
import folium
from folium.plugins import HeatMap
import numpy as np

metro_path_in = '../data/job_growth_data.csv'
metro_path_out = '../data/job_growth_data_w_lat_long.csv'
zip_path = '../data/zipcodes.csv'

In [11]:
# read in job growth data and get the lat and long for the metro area
df_job_growth = pd.read_csv(metro_path_in)
geolocator = Nominatim()

lat = []
long = []

for i in range(0,len(df_job_growth)):
    location = geolocator.geocode(df_job_growth['Metro-Area'][i] + ' South Carolina')
    try:
        lat.append(location.latitude)
        long.append(location.longitude)
    except AttributeError as err:  
        lat.append(0)
        long.append(0)
        
df_job_growth['lat'] = lat
df_job_growth['long'] = long


In [12]:
# map the growth rate for metro area to each zip code
zip_df = pd.read_csv(zip_path)
len_df = len(zip_df)

job_growth_model = NearestNDInterpolator(df_job_growth[['lat','long']], df_job_growth['job_growth_per'])
job_growth_zip = []                                                                                    
for i in range(0,len_df):
    job_growth_zip.append(job_growth_model(zip_df['latitude'][i],zip_df['longitude'][i]))
zip_df['job_growth_per']=job_growth_zip
zip_df.to_csv(metro_path_out)

In [ ]:
zip_df.keys()